In [0]:
# Define variables
table_name = dbutils.widgets.get('table_name')

In [0]:
# Describe the history of the table
history_df = spark.sql(f'''DESCRIBE HISTORY {table_name}''')

# Get the most recent operation
latest_row = history_df.orderBy(history_df['version'].desc()).first()
latest_op = latest_row['operation']

if latest_op in ("WRITE", "MERGE", "UPDATE"):
	# Define actual max version
	max_version = spark.sql(f'''SELECT MAX(version) as a
	FROM (DESCRIBE HISTORY {table_name})''')
	max_version_value = max_version.collect()[0][0]
	change_type = spark.sql(f'''select _change_type
	from table_changes('{table_name}', {max_version_value})''').collect()[0][0]
else:
	# Find the most recent row with WRITE or UPDATE
	write_update_df = history_df.filter(history_df['operation'].isin(["WRITE", "UPDATE"]))
	most_recent_write_update_row = write_update_df.orderBy(write_update_df['version'].desc()).limit(1).collect()[0]
	most_recent_write_update_version = most_recent_write_update_row['version']
	# Display the row as a DataFrame
	change_type = spark.sql(f'''select _change_type
	from table_changes('{table_name}', {most_recent_write_update_version})''').collect()[0][0]

In [0]:
change_type

In [0]:
dbutils.jobs.taskValues.set(key="change_type", value=change_type)